In [1]:
import numpy as np
from matplotlib import pyplot as plt

from warper import simulate_2d_only_memory, simulate_2d_only_memory_anharmonic_2

from multiprocessing import Pool

import os, pickle
from tqdm.notebook import tqdm

import pandas as pd

In [2]:
%matplotlib qt

In [3]:
pool = Pool(4)

In [4]:
def get_RSH(A,B,g0,b,ka):
    o0 = np.sqrt(-A)
    mu = ka
    S = 1/g0/2/ka**2*(ka**2+o0**2+g0*ka+b**2)
    R = 1/g0/2/o0**2/ka**2 * (o0**2+ka**2)
    H = - 1/2/g0/ka**2*b
    return R,S,H, 2*(R*S+H**2)

def mean_with_err(arr, axis):
    return arr.mean(axis=axis), arr.std(axis=axis)/np.sqrt(arr.shape[axis])

In [5]:
# params - A,B,C,D,F,g0,b,ka

params = [
#     (-1,0,1,0,1),
#     (-1,0,1,1,1),
#     (-1,0,1,2,1),
#     (-1,0,1,3,1),
    
#     (-1,0.5,1,0,1),
#     (-1,0.5,1,1,1),
#     (-1,0.5,1,2,1),
#     (-1,0.5,1,3,1),
    
#     (3,0.4,1,0,1),
#     (3,0.4,1,1,1),
#     (3,0.4,1,2,1),
#     (3,0.4,1,3,1),

]

bs =  (0,1,2,3,4,5,6,8,10)#,12,14,16,18,20,22,24,26,28,30)
bs =  (0,3,6,10)#,12,14,16,18,20,22,24,26,28,30)
params += [(-1,0,0,0,0,1,b,1) for b in bs]
params += [(3,0.4,0,0,0,1,b,1) for b in bs]
params += [(2,0.14,0,0,0,1,b,1) for b in bs]

df = pd.DataFrame(params, columns=["A","B","C","D","F","g0","b","ka"])

In [6]:
def f(row):
    return pd.Series(get_RSH(row.A, row.B, row.g0, row.b, row.ka),index=("R","S","H","varL"))
df = pd.concat([df, df.apply(f, axis=1)], axis=1, sort=False)

<ipython-input-4-fd8d9f319383>:2: RuntimeWarning: invalid value encountered in sqrt
  o0 = np.sqrt(-A)


In [7]:
df

,A,B,C,D,F,g0,b,ka,R,S,H,varL
0,-1,0.00,0,0,0,1,0,1,1.0,1.5,-0.0,3.0
1,-1,0.00,0,0,0,1,3,1,1.0,6.0,-1.5,16.5
2,-1,0.00,0,0,0,1,6,1,1.0,19.5,-3.0,57.0
3,-1,0.00,0,0,0,1,10,1,1.0,51.5,-5.0,153.0
4,3,0.40,0,0,0,1,0,1,NaN,NaN,-0.0,NaN
5,3,0.40,0,0,0,1,3,1,NaN,NaN,-1.5,NaN
6,3,0.40,0,0,0,1,6,1,NaN,NaN,-3.0,NaN
7,3,0.40,0,0,0,1,10,1,NaN,NaN,-5.0,NaN
8,2,0.14,0,0,0,1,0,1,NaN,NaN,-0.0,NaN
9,2,0.14,0,0,0,1,3,1,NaN,NaN,-1.5,NaN


In [8]:
N=3000000
warmup=0
dt=0.001
samples=100
skip=500
runs=4
time = np.linspace(0,N//skip *dt, N//skip)

dump_to_disk = False

In [ ]:
results = []
for row in tqdm(df.itertuples(), total=len(df)):
#     if row.Index <40:
#         continue
    kwargs = dict(x0=np.zeros(samples),y0=np.zeros(samples),
                  vx0=np.zeros(samples),vy0=np.zeros(samples),
        N=N, samples=samples,
        dt=dt, warmup=warmup, skip=skip,
        A=row.A,B=row.B,C=row.C,D=row.D,F=row.F,
        gamma0=row.g0,b=row.b, kappa=row.ka)
    res = simulate_2d_only_memory_anharmonic_2(pool,runs,**kwargs)
    if dump_to_disk:
        np.save(f"results/{row.Index}", res)
    else:
        results.append(res[:4])

In [ ]:
results = np.array(results)

### Load dumped data

In [ ]:
import os
n_files = len(os.listdir("results"))

In [ ]:
np.load("results/1.npy").shape

In [ ]:
results = np.zeros((n_files, 4, 800, 5000))

In [ ]:
for i in tqdm(range(n_files)):
    results[i] = np.load(f"results/{i}.npy")[:4,:,-5000:]

In [ ]:
results[:,:,:,:].shape

### Start Analisys

In [ ]:
cov = np.einsum("iakj,ibkj->ijab", results[:,:4,:,:],results[:,:4,:,:],optimize=True)/(results.shape[2]-1)

In [ ]:
cov.shape

In [ ]:
plt.plot(cov.transpose(1,0,2,3).reshape(-1,cov.shape[0]*16)[::], alpha=0.2);
# plt.plot(cov[5,:,1,2]);

In [ ]:
stationari_treshold = 2000

In [ ]:
results.shape

In [ ]:
Rx,Ry, Sx, Sy = results[...,stationari_treshold:].var(axis=2).transpose(1,0,2)
Lx = results[:,0] * results[:,3]
Ly = results[:,1] * results[:,2]

Hx = Lx.mean(axis=1)
Hy = Ly.mean(axis=1)

In [ ]:
df["Rx"], df["Rx_err"]= mean_with_err(Rx,axis=-1)
df["Ry"], df["Ry_err"]= mean_with_err(Ry,axis=-1)
df["Sx"], df["Sx_err"]= mean_with_err(Sx,axis=-1)
df["Sy"], df["Sy_err"]= mean_with_err(Sy,axis=-1)
df["Hx"], df["Hx_err"]= mean_with_err(Hx,axis=-1)
df["Hy"], df["Hy_err"]= mean_with_err(Hy,axis=-1)

# df["Rx"], df["Rx_err"]= mean_with_err(Rx,axis=-1)


In [ ]:
df[sorted(df.columns)].round(4)

In [ ]:
gb=df.groupby("B")

plt.figure(figsize=(10,7), dpi=150)
plt.title("H vs b")
plt.xlabel("b");plt.ylabel("H")
plt.plot(gb.get_group(0).b, gb.get_group(0).H, ls=(0, (5, 10)), label="harmonic")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Hx, yerr=gp.Hx_err*10, capsize=7, marker="o", ls="none", label=f"A: {gp.A.mean():0.3f} B: {gp.B.mean():0.3f}")
# plt.xlim(-1,5)
# plt.ylim(-4,1)
plt.legend()

In [ ]:
gb=df.groupby("B")

plt.figure(figsize=(10,7), dpi=150)
plt.subplot(121)
plt.title("R vs b")
plt.xlabel("b");plt.ylabel("R")
plt.plot(gb.get_group(0).b, gb.get_group(0).R, ls=(0, (5, 10)), label="harmonic")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Rx, yerr=gp.Rx_err*10, capsize=7, marker="o", ls="none", label=f"A: {gp.A.mean():0.3f} B: {gp.B.mean():0.3f}")
plt.legend()

plt.subplot(122)
plt.title("S vs b")
plt.xlabel("b");plt.ylabel("S")
plt.plot(gb.get_group(0).b, gb.get_group(0).S, ls=(0, (5, 10)), label="harmonic")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Sx, yerr=gp.Sx_err*10, capsize=7, marker="o", ls="none", label=f"A: {gp.A.mean():0.3f} B: {gp.B.mean():0.3f}")
plt.legend()

In [ ]:
gb.groups

In [ ]:
dir(gb)

In [ ]:
R_sample = results.var(axis=2)[:,0] 
S_sample = results.var(axis=2)[:,2]

In [ ]:
L = (results[:,0] * results[:,3] - results[:,1] * results[:,2])
H_sample = L.mean(axis=1)/2

In [ ]:
H_sample.shape

In [ ]:
H_sample[:,-3000:].mean(axis=1)

In [ ]:
np.sqrt(H_sample[:,-3000:].var(axis=1)/3000)

In [ ]:
H

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(131)
plt.scatter(S, S_sample[..., -1])
plt.subplot(132)
plt.scatter(R, R_sample[..., -1])
plt.subplot(133)
plt.scatter(H, H_sample[..., -1])

In [ ]:
results.shape

In [ ]:
L.shape

In [ ]:
# T = results[:,2,:,:]
T = L/2
hlines = H

In [ ]:
T.shape[0]

In [ ]:
fig, axs = plt.subplots(1+T.shape[0]//2,2,figsize=(14,8),dpi=120)
for ax, xi,h in zip(axs.flatten(),T[:,:,:], hlines):
    ax.plot(time, xi.mean(axis=0))
    ax.hlines(y=h, xmin=0, xmax=max(time), colors="r")
# plt.show()

In [ ]:
L.shape

In [ ]:
plt.hist(L[0,:,-1], bins=100);

In [ ]:
L[1,:,-1].mean()

In [ ]:
sorted(L[1,:,-1])[1600]

In [ ]:
L.shape

In [ ]:
plt.figure(figsize=(14,8))
plt.plot(L[1,2,3000:])